In [1]:
import pandas as pd

# Paper = https://drive.google.com/file/d/1lUtlu1vdGMymo8lkOTl_cftfcPDJMk0X/view?usp=sharing

In [2]:
df = pd.read_csv('datasets/preprocessed_df.csv')
df

,hotel,is_canceled,lead_time,arrival_date_month,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,meal,country,...,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,kids,total_guests,is_family,total_stays
0,Resort Hotel,0,342,7,27,0,0,2,BB,PRT,...,0.00,0,0,Check-Out,2015-07-01,2015-07-01,0.0,2.0,0,0
1,Resort Hotel,0,400,7,27,0,0,2,BB,PRT,...,0.00,0,0,Check-Out,2015-07-01,2015-07-01,0.0,2.0,0,0
2,Resort Hotel,0,7,7,27,0,1,1,BB,GBR,...,75.00,0,0,Check-Out,2015-07-02,2015-07-01,0.0,1.0,0,1
3,Resort Hotel,0,13,7,27,0,1,1,BB,GBR,...,75.00,0,0,Check-Out,2015-07-02,2015-07-01,0.0,1.0,0,1
4,Resort Hotel,0,14,7,27,0,2,2,BB,GBR,...,98.00,0,1,Check-Out,2015-07-03,2015-07-01,0.0,2.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86998,City Hotel,0,23,8,35,2,5,2,BB,BEL,...,96.14,0,0,Check-Out,2017-09-06,2017-08-30,0.0,2.0,0,7
86999,City Hotel,0,102,8,35,2,5,3,BB,FRA,...,225.43,0,2,Check-Out,2017-09-07,2017-08-31,0.0,3.0,0,7
87000,City Hotel,0,34,8,35,2,5,2,BB,DEU,...,157.71,0,4,Check-Out,2017-09-07,2017-08-31,0.0,2.0,0,7
87001,City Hotel,0,109,8,35,2,5,2,BB,GBR,...,104.40,0,0,Check-Out,2017-09-07,2017-08-31,0.0,2.0,0,7
